In [ ]:
from typing import List, Dict
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    AIMessage, 
    HumanMessage, 
    SystemMessage, 
    ToolMessage
)
from langchain_core.tools.structured import StructuredTool
from langchain.tools import tool
from langchain_core.output_parsers.openai_tools import parse_tool_calls
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

**The Agent Class**

In [3]:
class Agent:
    def __init__(
            self, 
            name:str="AI Agent",
            role:str="Personal Assistant",
            instructions:str = "Help users with any question", 
            model:str="gpt-4o-mini",
            temperature:float=0.0,            
            tools:List[StructuredTool]=[]):
        
        self.name = name
        self.role = role
        self.instructions = instructions

        self.llm = ChatOpenAI(
            model=model,
            temperature=temperature,
        )

        self.tools = tools
        self.tool_map = {tool.name:tool for tool in tools}
        self.memory = [
            SystemMessage(
                content=f"You're {self.name}, your role is {self.role}, " 
                        f"and you need to {self.instructions} "
            ),
        ]
        
    def invoke(self, user_message:str):
        self.memory.append(HumanMessage(content=user_message))
        ai_message = self._invoke_llm()

        tool_calls = ai_message.additional_kwargs.get('tool_calls')
        if tool_calls:
            self._call_tools(tool_calls)
            self._invoke_llm()

        return self.memory[-1].content

    def _invoke_llm(self)->AIMessage:
        llm = self.llm.bind_tools(self.tools)
        ai_message = llm.invoke(self.memory)
        self.memory.append(ai_message)
        return ai_message

    def _call_tools(self, tool_calls:List[Dict]):
        parsed_tool_calls = parse_tool_calls(tool_calls)
        for tool_call in parsed_tool_calls:
            tool_call_id = tool_call['id']
            function_name = tool_call['name']
            arguments = tool_call['args']
            func = self.tool_map[function_name]
            result = func.invoke(arguments)
            tool_message = ToolMessage(
                content=result,
                name=function_name,
                tool_call_id=tool_call_id,
            )
            self.memory.append(tool_message)

In [4]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [5]:
agent = Agent(
    tools=[multiply]
)

In [6]:
agent.invoke("2 multiplied by 2")

'2 multiplied by 2 is 4.'

In [7]:
agent.memory

[SystemMessage(content="You're AI Agent, your role is Personal Assistant, and you need to Help users with any question ", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='2 multiplied by 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QO5qyZ0KtHYYgYU4aQyaJNaG', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 69, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0374a514-a9dc-487a-bd1a-6d06f7903ed6-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 2}, 